### 3.2.1.Интеграция SQLAlchemy с Flask.

Flask не имеет встроенного ORM.  
SQLAlchemy — самая популярная ORM для Python. Позволяет работать с БД через Python-объекты. Автоматизирует создание/изменение таблиц. Обеспечивает безопасность (защита от SQL-инъекций). Поддерживает PostgreSQL, MySQL, SQLite и другие СУБД. Flask-SQLAlchemy — официальное расширение Flask для работы с SQLAlchemy. Alembic (опционально) — для миграций базы данных.

**Модель** — Python-класс, который представляет таблицу в БД  
**Сессия** — область взаимодействия с БД (аналог транзакции)  
**Запрос** (Query) — объект для выборки данных

Структура проекта

[myblog/](https://github.com/AnatolyKuzmin/Python/blob/main/3_Веб-разработка_(Flask_Django)/myblog_SQLAlchemy)  
├── app.py          # Основное приложение  
├── models.py       # Модели данных  
├── config.py       # Конфигурация  
└── templates/      # Шаблоны (если нужно)  

Примеры:  
Создание записи  
```
from models import User, Post
from app import create_app

app = create_app()

with app.app_context():
    # Создаём пользователя
    new_user = User(username='john_doe', email='john@example.com')
    db.session.add(new_user)
    
    # Создаём пост
    new_post = Post(title='First Post', content='Hello World!', author=new_user)
    db.session.add(new_post)
    
    # Фиксируем изменения
    db.session.commit()
```
Чтение данных  
```
with app.app_context():
    # Получаем всех пользователей
    users = User.query.all()
    
    # Получаем пользователя по username
    user = User.query.filter_by(username='john_doe').first()
    
    # Получаем все посты пользователя
    posts = Post.query.filter_by(user_id=user.id).all()
```
Обновление записи  
```
with app.app_context():
    user = User.query.get(1)  # Получаем пользователя с id=1
    user.email = 'new_email@example.com'
    db.session.commit()
```
Удаление записи  
```
with app.app_context():
    post = Post.query.get(1)  # Получаем пост с id=1
    db.session.delete(post)
    db.session.commit()
```

!!! Важные замечания:  
- Все изменения должны завершаться db.session.commit()  
- При ошибках используйте db.session.rollback()  
- Все операции с БД должны выполняться в контексте приложения  
- Используйте with app.app_context(): если вне route-функций

### 3.2.2.Создание моделей и миграции.

### 3.2.3.CRUD операции.

### 3.2.4.Практика: расширение блога с базой данных.